<a href="https://colab.research.google.com/github/scottthomaswx/WarningPlotter/blob/main/Warning_Plotter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Needed Code

In [1]:
# @title
%%capture
!pip install geopandas
!pip install metpy
!pip install requests
!pip install cartopy
!pip install matplotlib
!pip install numpy
import matplotlib.pyplot as plt
import geopandas
from metpy.plots import MapPanel, PanelContainer, PlotGeometry
import requests
import pandas as pd
from metpy.plots import USCOUNTIES
import cartopy.feature as cfeat
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import numpy as np
!rm -rf sample_data

#Enter Information To Pull Needed Data

######Note: If North/South Distance much greater East/West Distance, Longitude range will need to be expanded to ensure full North/South Distance is plotted

######Note: If Plotting for Long Fused Hazards (e.g. Tropical/Wind/Winter) Set Start Time Before Hazard Issuance!

In [57]:
# @markdown # Start and End Date/Time Selection
# @markdown ## Enter in the format YYYY-MM-DDT:HH:MM

Begins = '2022-12-20T12:00' #@param {type:"string"}
Ends = '2022-12-26T12:00' #@param {type:"string"}

WestLong = -90 # @param {type:"number"}
EastLong = -82 # @param {type:"number"}
NorthLat = 45 # @param {type:"number"}
SouthLat = 40 # @param {type:"number"}

#Select Which Type of Event(s) You Wish To Create Plots For

In [70]:
#Plot SVR/FFW/TOR/PS/SMW And All LSRs
SevereAndHydroIBWOnly = False # @param {type:"boolean"}

#Plot SVR/FFW/TOR/PS/SMW/FAY/FAW And All LSRs Above
SevereAndHydroFull = False # @param {type:"boolean"}

#Plot SVR/TOR/SMW/SPS and Severe Wx LSRs Only
SevereOnly = False # @param {type:"boolean"}

#Plot FFW/FLW/FAY and Rain/Flooding LSRs Only
HydroOnly = False # @param {type:"boolean"}

#Plot Non Convective Wind Alerts/LSRs Only
NonTstmWind = False # @param {type:"boolean"}

#Plot TS/HU/EWW !!Set Start Date to Issuance Date of First Hazards!!!
TropicalOnly = False # @param {type:"boolean"}

#Plot TS/HU/Wind !!Set Start Date to Issuance Date of First Hazards!!!
TropicalWind = False # @param {type:"boolean"}

#Plot All Wind/TOR/Flood !!Set Start Date to Issuance Date of First Hazards!!!
TropicalFull = False # @param {type:"boolean"}

#Plot Snow Squalls !!Set Start Date to Issuance Date of First Hazards!!!
SnowSquall = True # @param {type:"boolean"}

#Plot All Winter !!Set Start Date to Issuance Date of First Hazards!!!
WinterStorm = True # @param {type:"boolean"}


#Pull Warnings and LSRs

In [71]:
# @title
%%capture
#Pull Warnings
data = {'TYPECODE': [],
        'MAG': [],
        'PHENOM': [],
        'SIG': [],
        'phenomena': [],
        'geometry': []}

#Pull LSRs
try:
  LSR1 = 'https://mesonet.agron.iastate.edu/cgi-bin/request/gis/lsr.py?sts='+Begins+'Z&ets='+Ends+'Z&fmt=csv'
  LSR1File = pd.read_csv(LSR1, on_bad_lines='skip')
except:
  LSR1File = pd.DataFrame(data)

#Pull Warnings

if SevereAndHydroIBWOnly == True or SevereAndHydroFull == True or SevereOnly == True or HydroOnly == True or TropicalFull == True:
  try:
    Warn1 = geopandas.read_file('https://mesonet.agron.iastate.edu/api/1/cow.json?begints='+Begins+':00Z&endts='+Ends+':00Z&hailsize=1&wind=58&phenomena=TO&phenomena=SV&phenomena=MA&phenomena=FF&phenomena=DS&lsrtype=T&lsrtype=D&lsrtype=H&lsrtype=G&lsrtype=MA&lsrtype=FF&lsrtype=DS&lsrbuffer=15&warningbuffer=1')
  except:
    Warn1 = pd.DataFrame(data)
else:
  Warn1 = pd.DataFrame(data)

if SevereAndHydroIBWOnly == True or SevereAndHydroFull == True or SevereOnly == True:
  try:
    SPSFile = geopandas.read_file('https://mesonet.agron.iastate.edu/cgi-bin/request/gis/sps.py?&sts='+Begins+'Z&ets='+Ends+'Z&fmt=csv')
  except:
    SPSFile = pd.DataFrame(data)
else:
  SPSFile = pd.DataFrame(data)

if SevereAndHydroFull == True or HydroOnly == True:
  try:
    floodfile = geopandas.read_file(f'https://mesonet.agron.iastate.edu/cgi-bin/request/gis/watchwarn.py?accept=shapefile&sts={Begins}Z&ets={Ends}Z&limitps=yes&phenomena=FA,FA&significance=W,Y')
  except:
    pass
else:
  floodfile = pd.DataFrame(data)

if NonTstmWind == True or TropicalWind == True:
  try:
    windfile = geopandas.read_file(f'https://mesonet.agron.iastate.edu/cgi-bin/request/gis/watchwarn.py?accept=shapefile&sts={Begins}Z&ets={Ends}Z&limitps=yes&phenomena=HW,WI&significance=W,Y')
  except:
    windfile = pd.DataFrame(data)
else:
  windfile = pd.DataFrame(data)

if TropicalOnly == True or TropicalWind == True:
  try:
    tropfile = geopandas.read_file(f'https://mesonet.agron.iastate.edu/cgi-bin/request/gis/watchwarn.py?accept=shapefile&sts={Begins}Z&ets={Ends}Z&limitps=yes&phenomena=HU,TR,EW&significance=W,W,W')
  except:
    tropfile = pd.DataFrame(data)
else:
  tropfile = pd.DataFrame(data)

if SnowSquall == True or WinterStorm == True:
  try:
    snowsqfile = geopandas.read_file('https://mesonet.agron.iastate.edu/api/1/cow.json?begints='+Begins+':00Z&endts='+Ends+':00Z&phenomena=SQ&lsrtype=q&lsrbuffer=15&warningbuffer=1')
  except:
    snowsqfile = pd.DataFrame(data)
else:
  snowsqfile = pd.DataFrame(data)

if WinterStorm == True:
  try:
    winterstormfile = geopandas.read_file(f'https://mesonet.agron.iastate.edu/cgi-bin/request/gis/watchwarn.py?accept=shapefile&sts={Begins}Z&ets={Ends}Z&limitps=yes&phenomena=WS,SQ,IS,LE,BZ,WW&significance=W,W,W,W,W,Y')
  except:
    winterstorm = pd.DataFrame(data)
else:
  snowsqfile = pd.DataFrame(data)

#Add Marine Zones for Great Lakes
mzone_file = geopandas.read_file('/vsicurl/https://github.com/scottthomaswx/WarningPlotter/raw/main/mz19se23.shp')


#Parse Warnings and LSRs

In [72]:
# @title
TORWarn = Warn1.query('phenomena == "TO"')
SVRWarn = Warn1.query('phenomena == "SV"')
FFWWarn = Warn1.query('phenomena == "FF"')
SMWWarn = Warn1.query('phenomena == "MA"')
DSWWarn = Warn1.query('phenomena == "DS"')
SPSWarn = SPSFile
FAWWarn=floodfile.query('SIG == "W"')
FAYWarn=floodfile.query('SIG == "Y"')
HUWarn=tropfile.query('PHENOM == "HU"')
TRWarn=tropfile.query('PHENOM == "TR"')
EWWarn=tropfile.query('PHENOM == "EW"')

HWWWarn=windfile.query('PHENOM == "HW"')
WIYWarn=windfile.query('PHENOM == "WI"')

SQWWarn=snowsqfile.query('phenomena == "SQ"')
ISWWarn=winterstormfile.query('PHENOM == "IS"')
LEWWarn=winterstormfile.query('PHENOM == "LE"')
BZWWarn=winterstormfile.query('PHENOM == "BZ"')
WWYWarn=winterstormfile.query('PHENOM == "WW"')
WSWWarn=winterstormfile.query('PHENOM == "WS"')

LSR1File['MAG'] = LSR1File['MAG'].replace('None', np.nan)
LSR1File['MAG'] = LSR1File['MAG'].astype(float)

TORSR = LSR1File.query('TYPECODE == "T"')
FNCSR = LSR1File.query('TYPECODE == "C"')
SSTSGSR = LSR1File.query('TYPECODE == "G" & MAG >= 40 & MAG < 58')
TSGSR = LSR1File.query('TYPECODE == "G" & MAG >= 58 & MAG < 75')
SIGTSGSR = LSR1File.query('TYPECODE == "G" & MAG >= 75')
SSHALSR = LSR1File.query('TYPECODE == "H" & MAG < 1.0')
HALSR = LSR1File.query('TYPECODE == "H" & MAG >= 1.0 & MAG < 2.0')
SIGHALSR = LSR1File.query('TYPECODE == "H" & MAG >= 2.0')
TSDSR = LSR1File.query('TYPECODE == "D"')
SSMTGSR = LSR1File.query('TYPECODE == "M" & MAG >= 40 & MAG < 58')
MTGSR = LSR1File.query('TYPECODE == "M" & MAG >= 58 & MAG < 75')
SIGMTGSR = LSR1File.query('TYPECODE == "M" & MAG >= 75')
WSTSR = LSR1File.query('TYPECODE == "W"')

RANSR = LSR1File.query('TYPECODE == "R"')
FLDSR = LSR1File.query('TYPECODE == "E"')
FFDSR = LSR1File.query('TYPECODE == "F"')

LTGSR = LSR1File.query('TYPECODE == "L"')

NTYSR = LSR1File.query('TYPECODE == "N" & MAG >= 40 & MAG < 58')
NTWSR = LSR1File.query('TYPECODE == "N" & MAG >= 58 & MAG < 75')
NTSSR = LSR1File.query('TYPECODE == "N" & MAG >= 75 ')
NTDSR = LSR1File.query('TYPECODE == "O"')

SNW0SR = LSR1File.query('TYPECODE == "S" & MAG >= 0.0 & MAG < 1')
SNW1SR = LSR1File.query('TYPECODE == "S" & MAG >= 1.0 & MAG < 2')
SNW2SR = LSR1File.query('TYPECODE == "S" & MAG >= 2.0 & MAG < 4')
SNW6SR = LSR1File.query('TYPECODE == "S" & MAG >= 4.0 & MAG < 6')
SNW8SR = LSR1File.query('TYPECODE == "S" & MAG >= 6.0 & MAG < 8')
SNW12SR = LSR1File.query('TYPECODE == "S" & MAG >= 8.0 & MAG < 12')
SNW18SR = LSR1File.query('TYPECODE == "S" & MAG >= 12.0 & MAG < 18')
SNW24SR = LSR1File.query('TYPECODE == "S" & MAG >= 18.0 & MAG < 24')
SNW36SR = LSR1File.query('TYPECODE == "S" & MAG >= 24.0 & MAG < 36')
SNW48SR = LSR1File.query('TYPECODE == "S" & MAG >= 36.0 & MAG < 48')
SNW60SR = LSR1File.query('TYPECODE == "S" & MAG >= 48.0')

FZR0SR = LSR1File.query('TYPECODE == "5" & MAG >= 0.0 & MAG < 0.1')
FZR1SR = LSR1File.query('TYPECODE == "5" & MAG >= 0.1 & MAG < 0.25')
FZR2SR = LSR1File.query('TYPECODE == "5" & MAG >= 0.25 & MAG < 0.5')
FZR5SR = LSR1File.query('TYPECODE == "5" & MAG >= 0.5 & MAG < 1')
FZR10SR = LSR1File.query('TYPECODE == "5" & MAG >= 1.0')


SLT0SR = LSR1File.query('TYPECODE == "s" & MAG >= 0.0 & MAG < 1')
SLT10SR = LSR1File.query('TYPECODE == "s" & MAG >= 1.0 & MAG < 2')
SLT20SR = LSR1File.query('TYPECODE == "s" & MAG >= 2.0 & MAG < 4')
SLT40SR = LSR1File.query('TYPECODE == "s" & MAG >= 4.0')

BLZSR = LSR1File.query('TYPECODE == "Z"')

TRPSR = LSR1File.query('TYPECODE == "0"')

SQWSR = LSR1File.query('TYPECODE == "q"')

MarZoneMap = PlotGeometry()
MarZoneMap.geometry = mzone_file['geometry']
Warngeo = PlotGeometry()
Warngeo.geometry = Warn1['geometry']

#Create Image

In [73]:
# @title
%%capture
if SevereAndHydroIBWOnly == True:
  fig = plt.figure(figsize=(12, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  axb = ax.twinx()
  axb.set_box_aspect(0.7)
  #Add Geometries
  try:
    ax.add_geometries(TORWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'red', edgecolor = 'black', zorder = 998, alpha = 0.7)
    ax.add_geometries(SVRWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'yellow', edgecolor = 'black', zorder = 997, alpha = 0.7)
    ax.add_geometries(SMWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'orange', edgecolor = 'black', zorder = 996, alpha = 0.7)
    ax.add_geometries(FFWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'green', edgecolor = 'black', zorder = 995, alpha = 0.7)
    ax.add_geometries(SPSWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#FFE4B5', edgecolor = 'black', zorder = 993, alpha = 0.6)
  except:
    pass
  ax.add_geometries(MarZoneMap.geometry, crs = ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black',  zorder=999)

  #Plot LSRs
  ax.scatter(y = TORSR['LAT'], x = TORSR['LON'], zorder=1001, color = 'red', marker = 'v', edgecolor = 'black', label = 'Tornado', s = 55)
  ax.scatter(y = FNCSR['LAT'], x = FNCSR['LON'], zorder=999, color = 'hotpink', label = 'Funnel Cloud', s = 50, edgecolor = 'black')
  ax.scatter(y = SSTSGSR['LAT'], x = SSTSGSR['LON'], zorder=999, color = 'lightseagreen', label = 'TSTM Gust (40-58mph)', s = 50, edgecolor = 'black')
  ax.scatter(y = TSGSR['LAT'], x = TSGSR['LON'], zorder=1000, color = 'dodgerblue', label = 'TSTM Gust (58-74mph)', s = 50, edgecolor = 'black')
  ax.scatter(y = SIGTSGSR['LAT'], x = SIGTSGSR['LON'], zorder=1001, color = 'dodgerblue', label = 'SIG TSTM Gust (>75mph)', marker = 's', edgecolor = 'black', s = 55)
  ax.scatter(y = SSHALSR['LAT'], x = SSHALSR['LON'], zorder=999, color = 'olivedrab', label = 'Hail (< 1")', s = 50, edgecolor = 'black')
  ax.scatter(y = HALSR['LAT'], x = HALSR['LON'], zorder=1000, color = 'limegreen', label = 'Hail (1"-1.99")', s = 50, edgecolor = 'black')
  ax.scatter(y = SIGHALSR['LAT'], x = SIGHALSR['LON'], zorder=1001, color = 'limegreen', label = 'SIG Hail (>2")', marker = '^', edgecolor = 'black', s = 55)
  ax.scatter(y = SSMTGSR['LAT'], x = SSMTGSR['LON'], zorder=999, color = 'lightseagreen', s = 50, edgecolor = 'black')
  ax.scatter(y = MTGSR['LAT'], x = MTGSR['LON'], zorder=999, color = 'dodgerblue', s = 50, edgecolor = 'black')
  ax.scatter(y = SIGMTGSR['LAT'], x = SIGMTGSR['LON'], zorder=999, color = 'dodgerblue', s = 50, edgecolor = 'black', marker = 's',)
  ax.scatter(y = TSDSR['LAT'], x = TSDSR['LON'], zorder=1000, color = 'blue', label = 'TSTM Damage', s = 50, edgecolor = 'black')
  ax.scatter(y = LTGSR['LAT'], x = LTGSR['LON'], zorder=999, color = 'black', label = 'Lightining', s = 50, edgecolor = 'black')
  ax.scatter(y = RANSR['LAT'], x = RANSR['LON'], zorder=999, color = 'purple', label = 'Rain', s = 50, edgecolor = 'black')
  ax.scatter(y = FLDSR['LAT'], x = FLDSR['LON'], zorder=999, color = 'green', label = 'Flood', s = 50, edgecolor = 'black')
  ax.scatter(y = FFDSR['LAT'], x = FFDSR['LON'], zorder=999, color = 'olive', label = 'Flash Flood', s = 50, edgecolor = 'black')
  ax.scatter(y = WSTSR['LAT'], x = WSTSR['LON'], zorder=1000, color = 'salmon', label = 'Landspout/Waterspout', s = 50, edgecolor = 'black')

  #Create Legend
  TORW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='red')
  SMW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='orange')
  SVRW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='yellow')
  FFW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='green')
  SPSW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#FFE4B5')

  axb.legend([TORW, SVRW, SMW, FFW, SPSW], ['Tornado', 'Severe T-storm', 'Special Marine', 'Flash Flood', 'Special Wx Statement'],
              loc='upper left', fontsize=10, facecolor='white', markerscale=8,
              labelcolor='k', title='  Warning Type', title_fontsize=10,
            bbox_to_anchor=(1.10, 0.4), fancybox=True, shadow=True,).set_zorder(10)

  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)
  plt.figtext(0.11, 0.23, f'(c) SCOTT THOMAS | SOURCE: Iowa State', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)
  plt.figtext(0.45, 0.73, f'Warnings/LSRs Valid {Begins} to {Ends}', ha='center', weight='bold', fontsize=16)
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl.x_labels_top = False
  gl.x_labels_bottom = True
  axb.grid(False)
  axb.set_xticks([])
  axb.set_yticks([])
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1 .height*0.95])
  ax.legend(loc='lower left', bbox_to_anchor=(1.10, 0.4),
      ncol=1, fancybox=True, shadow=True, title = 'Local Storm Report Type')
  plt.savefig(f'FLD_SVR_IBWOnly_WarningsLSRs{Ends}.png', bbox_inches = 'tight')

if SevereAndHydroFull == True:
  fig = plt.figure(figsize=(12, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  axb = ax.twinx()
  axb.set_box_aspect(0.7)
  #Add Geometries
  try:
    ax.add_geometries(TORWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'red', edgecolor = 'black', zorder = 998, alpha = 0.7)
    ax.add_geometries(SVRWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'yellow', edgecolor = 'black', zorder = 997, alpha = 0.7)
    ax.add_geometries(SMWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'orange', edgecolor = 'black', zorder = 996, alpha = 0.7)
    ax.add_geometries(FFWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'green', edgecolor = 'black', zorder = 995, alpha = 0.7)
    ax.add_geometries(SPSWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#FFE4B5', edgecolor = 'black', zorder = 993, alpha = 0.6)
    ax.add_geometries(FAWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#00FF00', edgecolor = 'black', zorder = 992, alpha = 0.6)
    ax.add_geometries(FAYWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#00FF7F', edgecolor = 'black', zorder = 991, alpha = 0.6)
  except:
    pass
  ax.add_geometries(MarZoneMap.geometry, crs = ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black',  zorder=999)

  #Plot LSRs
  ax.scatter(y = TORSR['LAT'], x = TORSR['LON'], zorder=1001, color = 'red', marker = 'v', edgecolor = 'black', label = 'Tornado', s = 55)
  ax.scatter(y = FNCSR['LAT'], x = FNCSR['LON'], zorder=999, color = 'hotpink', label = 'Funnel Cloud', s = 50, edgecolor = 'black')
  ax.scatter(y = SSTSGSR['LAT'], x = SSTSGSR['LON'], zorder=999, color = 'lightseagreen', label = 'TSTM Gust (40-58mph)', s = 50, edgecolor = 'black')
  ax.scatter(y = TSGSR['LAT'], x = TSGSR['LON'], zorder=1000, color = 'dodgerblue', label = 'TSTM Gust (58-74mph)', s = 50, edgecolor = 'black')
  ax.scatter(y = SIGTSGSR['LAT'], x = SIGTSGSR['LON'], zorder=1001, color = 'dodgerblue', label = 'SIG TSTM Gust (>75mph)', marker = 's', edgecolor = 'black', s = 55)
  ax.scatter(y = SSHALSR['LAT'], x = SSHALSR['LON'], zorder=999, color = 'olivedrab', label = 'Hail (< 1")', s = 50, edgecolor = 'black')
  ax.scatter(y = HALSR['LAT'], x = HALSR['LON'], zorder=1000, color = 'limegreen', label = 'Hail (1"-1.99")', s = 50, edgecolor = 'black')
  ax.scatter(y = SIGHALSR['LAT'], x = SIGHALSR['LON'], zorder=1001, color = 'limegreen', label = 'SIG Hail (>2")', marker = '^', edgecolor = 'black', s = 55)
  ax.scatter(y = SSMTGSR['LAT'], x = SSMTGSR['LON'], zorder=999, color = 'lightseagreen', s = 50, edgecolor = 'black')
  ax.scatter(y = MTGSR['LAT'], x = MTGSR['LON'], zorder=999, color = 'dodgerblue', s = 50, edgecolor = 'black')
  ax.scatter(y = SIGMTGSR['LAT'], x = SIGMTGSR['LON'], zorder=999, color = 'dodgerblue', s = 50, edgecolor = 'black', marker = 's',)
  ax.scatter(y = TSDSR['LAT'], x = TSDSR['LON'], zorder=1000, color = 'blue', label = 'TSTM Damage', s = 50, edgecolor = 'black')
  ax.scatter(y = LTGSR['LAT'], x = LTGSR['LON'], zorder=999, color = 'black', label = 'Lightining', s = 50, edgecolor = 'black')
  ax.scatter(y = RANSR['LAT'], x = RANSR['LON'], zorder=999, color = 'purple', label = 'Rain', s = 50, edgecolor = 'black')
  ax.scatter(y = FLDSR['LAT'], x = FLDSR['LON'], zorder=999, color = 'green', label = 'Flood', s = 50, edgecolor = 'black')
  ax.scatter(y = FFDSR['LAT'], x = FFDSR['LON'], zorder=999, color = 'olive', label = 'Flash Flood', s = 50, edgecolor = 'black')
  ax.scatter(y = WSTSR['LAT'], x = WSTSR['LON'], zorder=1000, color = 'salmon', label = 'Landspout/Waterspout', s = 50, edgecolor = 'black')

  TORW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='red')
  SMW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='orange')
  SVRW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='yellow')
  FFW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='green')
  SPSW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#FFE4B5')
  FAWW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#00FF00')
  FAYW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#00FF7F')

  axb.legend([TORW, SVRW, SMW, FFW, SPSW, FAWW, FAYW], ['Tornado', 'Severe T-storm', 'Special Marine', 'Flash Flood', 'Special Wx Statement', 'Areal Flood Warning', 'Areal Flood Advisory'],
              loc='upper left', fontsize=10, facecolor='white', markerscale=8,
              labelcolor='k', title='  Warning Type', title_fontsize=10,
            bbox_to_anchor=(1.10, 0.4), fancybox=True, shadow=True,).set_zorder(10)

  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)
  plt.figtext(0.11, 0.23, f'(c) SCOTT THOMAS | SOURCE: Iowa State', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)
  plt.figtext(0.45, 0.73, f'Warnings/LSRs Valid {Begins} to {Ends}', ha='center', weight='bold', fontsize=16)
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl.x_labels_top = False
  gl.x_labels_bottom = True
  axb.grid(False)
  axb.set_xticks([])
  axb.set_yticks([])
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1 .height*0.95])
  ax.legend(loc='lower left', bbox_to_anchor=(1.10, 0.4),
      ncol=1, fancybox=True, shadow=True, title = 'Local Storm Report Type')
  plt.savefig(f'FLD_SVR_All_WarningsLSRs{Ends}.png', bbox_inches = 'tight')

if SevereOnly == True:
  fig = plt.figure(figsize=(12, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  axb = ax.twinx()
  axb.set_box_aspect(0.7)
  #Add Geometries
  try:
    ax.add_geometries(TORWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'red', edgecolor = 'black', zorder = 998, alpha = 0.7)
    ax.add_geometries(SVRWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'yellow', edgecolor = 'black', zorder = 997, alpha = 0.7)
    ax.add_geometries(SMWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'orange', edgecolor = 'black', zorder = 996, alpha = 0.7)
    ax.add_geometries(FFWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'green', edgecolor = 'black', zorder = 995, alpha = 0.7)
    ax.add_geometries(SPSWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#FFE4B5', edgecolor = 'black', zorder = 993, alpha = 0.6)
  except:
    pass
  ax.add_geometries(MarZoneMap.geometry, crs = ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black',  zorder=999)

  #Plot LSRs
  ax.scatter(y = TORSR['LAT'], x = TORSR['LON'], zorder=1001, color = 'red', marker = 'v', edgecolor = 'black', label = 'Tornado', s = 55)
  ax.scatter(y = FNCSR['LAT'], x = FNCSR['LON'], zorder=999, color = 'hotpink', label = 'Funnel Cloud', s = 50, edgecolor = 'black')
  ax.scatter(y = SSTSGSR['LAT'], x = SSTSGSR['LON'], zorder=999, color = 'lightseagreen', label = 'TSTM Gust (40-58mph)', s = 50, edgecolor = 'black')
  ax.scatter(y = TSGSR['LAT'], x = TSGSR['LON'], zorder=1000, color = 'dodgerblue', label = 'TSTM Gust (58-74mph)', s = 50, edgecolor = 'black')
  ax.scatter(y = SIGTSGSR['LAT'], x = SIGTSGSR['LON'], zorder=1001, color = 'dodgerblue', label = 'SIG TSTM Gust (>75mph)', marker = 's', edgecolor = 'black', s = 55)
  ax.scatter(y = SSHALSR['LAT'], x = SSHALSR['LON'], zorder=999, color = 'olivedrab', label = 'Hail (< 1")', s = 50, edgecolor = 'black')
  ax.scatter(y = HALSR['LAT'], x = HALSR['LON'], zorder=1000, color = 'limegreen', label = 'Hail (1"-1.99")', s = 50, edgecolor = 'black')
  ax.scatter(y = SIGHALSR['LAT'], x = SIGHALSR['LON'], zorder=1001, color = 'limegreen', label = 'SIG Hail (>2")', marker = '^', edgecolor = 'black', s = 55)
  ax.scatter(y = SSMTGSR['LAT'], x = SSMTGSR['LON'], zorder=999, color = 'lightseagreen', s = 50, edgecolor = 'black')
  ax.scatter(y = MTGSR['LAT'], x = MTGSR['LON'], zorder=999, color = 'dodgerblue', s = 50, edgecolor = 'black')
  ax.scatter(y = SIGMTGSR['LAT'], x = SIGMTGSR['LON'], zorder=999, color = 'dodgerblue', s = 50, edgecolor = 'black', marker = 's',)
  ax.scatter(y = TSDSR['LAT'], x = TSDSR['LON'], zorder=1000, color = 'blue', label = 'TSTM Damage', s = 50, edgecolor = 'black')
  ax.scatter(y = WSTSR['LAT'], x = WSTSR['LON'], zorder=1000, color = 'salmon', label = 'Landspout/Waterspout', s = 50, edgecolor = 'black')

  #Create Legend
  TORW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='red')
  SMW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='orange')
  SVRW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='yellow')
  SPSW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#FFE4B5')

  axb.legend([TORW, SVRW, SMW, SPSW], ['Tornado', 'Severe T-storm', 'Special Marine', 'Special Wx Statement'],
              loc='upper left', fontsize=10, facecolor='white', markerscale=8,
              labelcolor='k', title='  Warning Type', title_fontsize=10,
            bbox_to_anchor=(1.10, 0.4), fancybox=True, shadow=True,).set_zorder(10)

  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)
  plt.figtext(0.11, 0.23, f'(c) SCOTT THOMAS | SOURCE: Iowa State', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)
  plt.figtext(0.45, 0.73, f'Warnings/LSRs Valid {Begins} to {Ends}', ha='center', weight='bold', fontsize=16)
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl.x_labels_top = False
  gl.x_labels_bottom = True
  axb.grid(False)
  axb.set_xticks([])
  axb.set_yticks([])
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1 .height*0.95])
  ax.legend(loc='lower left', bbox_to_anchor=(1.10, 0.4),
      ncol=1, fancybox=True, shadow=True, title = 'Local Storm Report Type')
  plt.savefig(f'SVR_WarningsLSRs{Ends}.png', bbox_inches = 'tight')

if HydroOnly == True:
  fig = plt.figure(figsize=(12, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  axb = ax.twinx()
  axb.set_box_aspect(0.7)
  #Add Geometries
  try:
    ax.add_geometries(FFWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  'green', edgecolor = 'black', zorder = 995, alpha = 0.7)
    ax.add_geometries(FAWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#00FF00', edgecolor = 'black', zorder = 992, alpha = 0.6)
    ax.add_geometries(FAYWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#00FF7F', edgecolor = 'black', zorder = 991, alpha = 0.6)
  except:
    pass
  ax.add_geometries(MarZoneMap.geometry, crs = ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black',  zorder=999)

  #Plot LSRs
  ax.scatter(y = RANSR['LAT'], x = RANSR['LON'], zorder=997, color = 'purple', label = 'Rain', s = 50, edgecolor = 'black')
  ax.scatter(y = FLDSR['LAT'], x = FLDSR['LON'], zorder=998, color = 'green', label = 'Flood', s = 50, edgecolor = 'black')
  ax.scatter(y = FFDSR['LAT'], x = FFDSR['LON'], zorder=999, color = 'olive', label = 'Flash Flood', s = 50, edgecolor = 'black')

  #Create Legend
  FFW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='green')
  FAWW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#00FF00')
  FAYW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#00FF7F')

  axb.legend([FFW, FAWW, FAYW], ['Flash Flood','Areal Flood Warning', 'Areal Flood Advisory'],
              loc='upper left', fontsize=10, facecolor='white', markerscale=8,
              labelcolor='k', title='  Warning Type', title_fontsize=10,
            bbox_to_anchor=(1.10, 0.4), fancybox=True, shadow=True,).set_zorder(10)

  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)
  plt.figtext(0.11, 0.23, f'(c) SCOTT THOMAS | SOURCE: Iowa State', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)
  plt.figtext(0.45, 0.73, f'Warnings/LSRs Valid {Begins} to {Ends}', ha='center', weight='bold', fontsize=16)
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl.x_labels_top = False
  gl.x_labels_bottom = True
  axb.grid(False)
  axb.set_xticks([])
  axb.set_yticks([])
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1 .height*0.95])
  ax.legend(loc='lower left', bbox_to_anchor=(1.10, 0.4),
      ncol=1, fancybox=True, shadow=True, title = 'Local Storm Report Type')
  plt.savefig(f'FLD_WarningsLSRs{Ends}.png', bbox_inches = 'tight')

if NonTstmWind == True:
  fig = plt.figure(figsize=(12, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  axb = ax.twinx()
  axb.set_box_aspect(0.7)
  #Add Geometries
  try:
    ax.add_geometries(HWWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#DAA520', edgecolor = 'black', zorder = 995, alpha = 1)
    ax.add_geometries(WIYWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#D2B48C', edgecolor = 'black', zorder = 992, alpha = 1)
  except:
    pass
  ax.add_geometries(MarZoneMap.geometry, crs = ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black',  zorder=999)

  #Plot LSRs
  ax.scatter(y = NTYSR['LAT'], x = NTYSR['LON'], zorder=999, color = 'lightseagreen', label = 'Non TSTM Wind Gust (40-57 mph)', s = 50, edgecolor = 'black')
  ax.scatter(y = NTWSR['LAT'], x = NTWSR['LON'], zorder=999, color = 'dodgerblue', label = 'Non TSTM Wind Gust (>58 mph)', s = 50, edgecolor = 'black')
  ax.scatter(y = NTSSR['LAT'], x = NTSSR['LON'], zorder=1001, color = 'dodgerblue', label = 'Non TSTM Gust (>75mph)', marker = 's', edgecolor = 'black', s = 55)
  ax.scatter(y = NTDSR['LAT'], x = NTDSR['LON'], zorder=999, color = 'blue', label = 'Non TSTM Wind Damage', s = 50, edgecolor = 'black')


  #Create Legend
  HWWW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#DAA520')
  WIYW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#D2B48C')

  axb.legend([HWWW, WIYW], ['High Wind Warning','Wind Advisory'],
              loc='upper left', fontsize=10, facecolor='white', markerscale=8,
              labelcolor='k', title='  Warning Type', title_fontsize=10,
            bbox_to_anchor=(1.10, 0.4), fancybox=True, shadow=True,).set_zorder(10)

  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)
  plt.figtext(0.11, 0.23, f'(c) SCOTT THOMAS | SOURCE: Iowa State', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)
  plt.figtext(0.45, 0.73, f'Warnings/LSRs Valid {Begins} to {Ends}', ha='center', weight='bold', fontsize=16)
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl.x_labels_top = False
  gl.x_labels_bottom = True
  axb.grid(False)
  axb.set_xticks([])
  axb.set_yticks([])
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1 .height*0.95])
  ax.legend(loc='lower left', bbox_to_anchor=(1.10, 0.4),
      ncol=1, fancybox=True, shadow=True, title = 'Local Storm Report Type')
  plt.savefig(f'WND_WarningsLSRs{Ends}.png', bbox_inches = 'tight')

if TropicalOnly == True:
  fig = plt.figure(figsize=(12, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  axb = ax.twinx()
  axb.set_box_aspect(0.7)
  #Add Geometries
  try:
    ax.add_geometries(HUWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#DC143C', edgecolor = 'black', zorder = 995, alpha = 1)
    ax.add_geometries(TRWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#B22222', edgecolor = 'black', zorder = 994, alpha = 1)
    ax.add_geometries(EWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#FF8C00', edgecolor = 'black', zorder = 996, alpha = 1)
  except:
    pass
  ax.add_geometries(MarZoneMap.geometry, crs = ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black',  zorder=999)

  #Plot LSRs
  ax.scatter(y = TRPSR['LAT'], x = TRPSR['LON'], zorder=999, color = 'Pink', label = 'Tropical Cyclone', s = 50, edgecolor = 'black')

  #Create Legend
  EXRW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#FF8C00')
  HURW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#DC143C')
  TSRW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#B22222')

  axb.legend([EXRW, HURW, TSRW], ['Extreme Wind Warning','Hurricane Warning', 'Tropical Storm Warning'],
              loc='upper left', fontsize=10, facecolor='white', markerscale=8,
              labelcolor='k', title='  Warning Type', title_fontsize=10,
            bbox_to_anchor=(1.10, 0.4), fancybox=True, shadow=True,).set_zorder(10)

  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)
  plt.figtext(0.11, 0.23, f'(c) SCOTT THOMAS | SOURCE: Iowa State', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)
  plt.figtext(0.45, 0.73, f'Warnings/LSRs Valid {Begins} to {Ends}', ha='center', weight='bold', fontsize=16)
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl.x_labels_top = False
  gl.x_labels_bottom = True
  axb.grid(False)
  axb.set_xticks([])
  axb.set_yticks([])
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1 .height*0.95])
  ax.legend(loc='lower left', bbox_to_anchor=(1.10, 0.4),
      ncol=1, fancybox=True, shadow=True, title = 'Local Storm Report Type')
  plt.savefig(f'TRP_WarningsLSRs{Ends}.png', bbox_inches = 'tight')

if TropicalWind == True:
  fig = plt.figure(figsize=(12, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  axb = ax.twinx()
  axb.set_box_aspect(0.7)
  #Add Geometries
  try:
    ax.add_geometries(HUWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#DC143C', edgecolor = 'black', zorder = 995, alpha = 1)
    ax.add_geometries(TRWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#B22222', edgecolor = 'black', zorder = 992, alpha = 1)
    ax.add_geometries(EWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#FF8C00', edgecolor = 'black', zorder = 996, alpha = 1)
    ax.add_geometries(HWWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#DAA520', edgecolor = 'black', zorder = 994, alpha = 1)
    ax.add_geometries(WIYWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#D2B48C', edgecolor = 'black', zorder = 991, alpha = 1)
  except:
    pass
  ax.add_geometries(MarZoneMap.geometry, crs = ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black',  zorder=999)

  #Plot LSRs
  ax.scatter(y = TRPSR['LAT'], x = TRPSR['LON'], zorder=999, color = 'Pink', label = 'Tropical Cyclone', s = 50, edgecolor = 'black')
  ax.scatter(y = NTYSR['LAT'], x = NTYSR['LON'], zorder=999, color = 'lightseagreen', label = 'Non TSTM Wind Gust (40-57 mph)', s = 50, edgecolor = 'black')
  ax.scatter(y = NTWSR['LAT'], x = NTWSR['LON'], zorder=999, color = 'dodgerblue', label = 'Non TSTM Wind Gust (>58 mph)', s = 50, edgecolor = 'black')
  ax.scatter(y = NTSSR['LAT'], x = NTSSR['LON'], zorder=1001, color = 'dodgerblue', label = 'Non TSTM Gust (>75mph)', marker = 's', edgecolor = 'black', s = 55)
  ax.scatter(y = NTDSR['LAT'], x = NTDSR['LON'], zorder=999, color = 'blue', label = 'Non TSTM Wind Damage', s = 50, edgecolor = 'black')
  ax.scatter(y = SSTSGSR['LAT'], x = SSTSGSR['LON'], zorder=999, color = 'lightseagreen', label = 'TSTM Gust (40-58mph)', s = 50, edgecolor = 'black')
  ax.scatter(y = TSGSR['LAT'], x = TSGSR['LON'], zorder=1000, color = 'dodgerblue', label = 'TSTM Gust (58-74mph)', s = 50, edgecolor = 'black')
  ax.scatter(y = SIGTSGSR['LAT'], x = SIGTSGSR['LON'], zorder=1001, color = 'dodgerblue', label = 'SIG TSTM Gust (>75mph)', marker = 's', edgecolor = 'black', s = 55)
  ax.scatter(y = TSDSR['LAT'], x = TSDSR['LON'], zorder=1000, color = 'blue', label = 'TSTM Damage', s = 50, edgecolor = 'black')

  #Create Legend
  EXRW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#FF8C00')
  HURW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#DC143C')
  TSRW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#B22222')
  HWWW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#DAA520')
  WIYW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#D2B48C')

  axb.legend([EXRW, HURW, TSRW, HWWW, WIYW], ['Extreme Wind Warning','Hurricane Warning', 'Tropical Storm Warning', 'High Wind Warning', 'Wind Advisory'],
              loc='upper left', fontsize=10, facecolor='white', markerscale=8,
              labelcolor='k', title='  Warning Type', title_fontsize=10,
            bbox_to_anchor=(1.10, 0.4), fancybox=True, shadow=True,).set_zorder(10)

  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)
  plt.figtext(0.11, 0.23, f'(c) SCOTT THOMAS | SOURCE: Iowa State', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)
  plt.figtext(0.45, 0.73, f'Warnings/LSRs Valid {Begins} to {Ends}', ha='center', weight='bold', fontsize=16)
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl.x_labels_top = False
  gl.x_labels_bottom = True
  axb.grid(False)
  axb.set_xticks([])
  axb.set_yticks([])
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1 .height*0.95])
  ax.legend(loc='lower left', bbox_to_anchor=(1.10, 0.4),
      ncol=1, fancybox=True, shadow=True, title = 'Local Storm Report Type')
  plt.savefig(f'TRP_Wind_WarningsLSRs{Ends}.png', bbox_inches = 'tight')

if SnowSquall == True:
  fig = plt.figure(figsize=(12, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  axb = ax.twinx()
  axb.set_box_aspect(0.7)
  #Add Geometries
  try:
    ax.add_geometries(SQWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#C71585', edgecolor = 'black', zorder = 998, alpha = 1)
    ax.add_geometries(SPSWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#FFE4B5', edgecolor = 'black', zorder = 993, alpha = 0.6)
  except:
    pass
  ax.add_geometries(MarZoneMap.geometry, crs = ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black',  zorder=999)

  #Plot LSRs
  ax.scatter(y = SQWSR['LAT'], x = SQWSR['LON'], zorder=999, color = 'Orange', label = 'Snow Squall', s = 50, edgecolor = 'black' , marker = 'D')

  #Create Legend
  SQWW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#C71585')
  SPSW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#FFE4B5')

  axb.legend([SQWW, SPSW], ['Snow Squall Warning', 'Special Wx Statement'],
              loc='upper left', fontsize=10, facecolor='white', markerscale=8,
              labelcolor='k', title='  Warning Type', title_fontsize=10,
            bbox_to_anchor=(1.10, 0.4), fancybox=True, shadow=True,).set_zorder(10)

  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)
  plt.figtext(0.11, 0.23, f'(c) SCOTT THOMAS | SOURCE: Iowa State', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)
  plt.figtext(0.45, 0.73, f'Warnings/LSRs Valid {Begins} to {Ends}', ha='center', weight='bold', fontsize=16)
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl.x_labels_top = False
  gl.x_labels_bottom = True
  axb.grid(False)
  axb.set_xticks([])
  axb.set_yticks([])
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1 .height*0.95])
  ax.legend(loc='lower left', bbox_to_anchor=(1.10, 0.4),
      ncol=1, fancybox=True, shadow=True, title = 'Local Storm Report Type')
  plt.savefig(f'SQW_Full_WarningsLSRs{Ends}.png', bbox_inches = 'tight')

if SnowSquall == True:
  fig = plt.figure(figsize=(12, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  axb = ax.twinx()
  axb.set_box_aspect(0.7)
  #Add Geometries
  try:
    ax.add_geometries(WSWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#FF69B4', edgecolor = 'black', zorder = 997, alpha = 1)
    ax.add_geometries(SQWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#C71585', edgecolor = 'black', zorder = 998, alpha = 1)
    ax.add_geometries(WWYWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#7B68EE', edgecolor = 'black', zorder = 996, alpha = 1)
    ax.add_geometries(ISWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#8B008B', edgecolor = 'black', zorder = 997, alpha = 1)
    ax.add_geometries(BZWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#FF4500', edgecolor = 'black', zorder = 998, alpha = 1)
    ax.add_geometries(LEWWarn.geometry, crs = ccrs.PlateCarree(), facecolor =  '#008B8B', edgecolor = 'black', zorder = 997, alpha = 1)
  except:
    pass
  ax.add_geometries(MarZoneMap.geometry, crs = ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black',  zorder=999)

  #Plot LSRs

  ax.scatter(y = SNW0SR['LAT'], x = SNW0SR['LON'], zorder=999, color = '#bdd7e6', label = 'Snow 0-1"', s = 50, edgecolor = 'black')
  ax.scatter(y = SNW1SR['LAT'], x = SNW1SR['LON'], zorder=999, color = '#66b0d6', label = 'Snow 1-2"', s = 50, edgecolor = 'black')
  ax.scatter(y = SNW2SR['LAT'], x = SNW2SR['LON'], zorder=999, color = '#08529a', label = 'Snow 2-4"', s = 50, edgecolor = 'black')
  ax.scatter(y = SNW6SR['LAT'], x = SNW6SR['LON'], zorder=999, color = '#082694', label = 'Snow 4-6"', s = 50, edgecolor = 'black')
  ax.scatter(y = SNW8SR['LAT'], x = SNW8SR['LON'], zorder=999, color = '#ffff95', label = 'Snow 6-8"', s = 50, edgecolor = 'black')
  ax.scatter(y = SNW12SR['LAT'], x = SNW12SR['LON'], zorder=999, color = '#fec400', label = 'Snow 8-12"', s = 50, edgecolor = 'black')
  ax.scatter(y = SNW18SR['LAT'], x = SNW18SR['LON'], zorder=999, color = '#ff8700', label = 'Snow 12-18"', s = 50, edgecolor = 'black')
  ax.scatter(y = SNW24SR['LAT'], x = SNW24SR['LON'], zorder=999, color = '#db1400', label = 'Snow 18-24"', s = 50, edgecolor = 'black')
  ax.scatter(y = SNW36SR['LAT'], x = SNW36SR['LON'], zorder=999, color = '#db1400', label = 'Snow 24-36"', s = 50, edgecolor = 'black')
  ax.scatter(y = SNW48SR['LAT'], x = SNW48SR['LON'], zorder=999, color = '#ff9999', label = 'Snow 36-48"', s = 50, edgecolor = 'black')
  ax.scatter(y = SNW60SR['LAT'], x = SNW60SR['LON'], zorder=999, color = 'black', label = 'Snow >=48""', s = 50, edgecolor = 'black')


  ax.scatter(y = FZR0SR['LAT'], x = FZR0SR['LON'], zorder=999, color = '#f4ea3b', label = 'Freezing Rain 0-0.1"', s = 50, edgecolor = 'black', marker = '^')
  ax.scatter(y = FZR1SR['LAT'], x = FZR1SR['LON'], zorder=999, color = '#ffc000', label = 'Freezing Rain 0.1-0.25"', s = 50, edgecolor = 'black', marker = '^')
  ax.scatter(y = FZR2SR['LAT'], x = FZR2SR['LON'], zorder=999, color = '#fe0000', label = 'Freezing Rain 0.25-0.5"', s = 50, edgecolor = 'black', marker = '^')
  ax.scatter(y = FZR5SR['LAT'], x = FZR5SR['LON'], zorder=999, color = '#c00000', label = 'Freezing Rain 0.5-1"', s = 50, edgecolor = 'black', marker = '^')
  ax.scatter(y = FZR10SR['LAT'], x = FZR10SR['LON'], zorder=999, color = '#730ac7', label = 'Freezing Rain >=1"', s = 50, edgecolor = 'black', marker = '^')

  ax.scatter(y = SLT0SR['LAT'], x = SLT0SR['LON'], zorder=999, color = '#bdd7e6', label = 'Sleet 0-1"', s = 50, edgecolor = 'black' , marker = 's')
  ax.scatter(y = SLT10SR['LAT'], x = SLT10SR['LON'], zorder=999, color = '#66b0d6', label = 'Sleet 1-2"', s = 50, edgecolor = 'black' , marker = 's')
  ax.scatter(y = SLT20SR['LAT'], x = SLT20SR['LON'], zorder=999, color = '#08529a', label = 'Sleet 2-4"', s = 50, edgecolor = 'black' , marker = 's')
  ax.scatter(y = SLT40SR['LAT'], x = SLT40SR['LON'], zorder=999, color = '#ffff95', label = 'Sleet >=4"', s = 50, edgecolor = 'black' , marker = 's')

  ax.scatter(y = BLZSR['LAT'], x = BLZSR['LON'], zorder=999, color = 'White', label = 'Blizzard', s = 50, edgecolor = 'black' , marker = 'D')
  ax.scatter(y = SQWSR['LAT'], x = SQWSR['LON'], zorder=999, color = 'Orange', label = 'Snow Squall', s = 50, edgecolor = 'black' , marker = 'D')

  #Create Legend
  SQWW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#C71585')
  WSWW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#66b0d6')
  WWYW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#7B68EE')
  LEWW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#008B8B')
  BZWW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#7B68EE')
  ISWW = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8B008B')
  axb.legend([SQWW, WSWW, WWYW, LEWW, BZWW, ISWW], ['Snow Squall Warning', "Winter Storm Warning", "Winter Weather Advisory", "Lake Effect Snow Warning", "Blizzard Warning", "Ice Storm Warning"],
              loc='upper left', fontsize=10, facecolor='white', markerscale=8,
              labelcolor='k', title='  Warning Type', title_fontsize=10,
            bbox_to_anchor=(1.10, 0.2), fancybox=True, shadow=True,).set_zorder(10)

  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)
  plt.figtext(0.11, 0.23, f'(c) SCOTT THOMAS | SOURCE: Iowa State', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)
  plt.figtext(0.45, 0.73, f'Warnings/LSRs Valid {Begins} to {Ends}', ha='center', weight='bold', fontsize=16)
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl.x_labels_top = False
  gl.x_labels_bottom = True
  axb.grid(False)
  axb.set_xticks([])
  axb.set_yticks([])
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1 .height*0.95])
  ax.legend(loc='lower left', bbox_to_anchor=(1.10, 0.2),
      ncol=1, fancybox=True, shadow=True, title = 'Local Storm Report Type')
  plt.savefig(f'WSW_Full_WarningsLSRs{Ends}.png', bbox_inches = 'tight')

#Clear All Data To Avoid Extraneous Variables For Future Plots

In [68]:
# @title
#Clear CWA 2/3/4 Warnings, SPSs, and LSRs for Additional Plots
try:
  del TORWarn, SVRWarn, FFWWarn, SMWWarn, DSWWarn, HWWWarn, WIYWarn, FAWWarn, FAYWarn, SPSWarn, HUWarn, TRWarn, EWWarn, SQWWarn, WSWWarn, WWYWarn, LEWWarn, BZWWarn, ISWWarn
except:
  pass
try:
  del TORSR, FNCSR, LTGSR, RANSR, HALSR, TSDSR, TSGSR, FLDSR, FFDSR, NTYSR, NTWSR, NTSSR, NTDSR, SIGTSGSR, SIGHALSR, TRPSR, BLZSR, SNW0SR, SNW1SR, SNW2SR, SNW6SR, SNW8SR, SNW12SR, SNW18SR, SNW24SR, SNW36SR, SNW48SR, SNW60SR, FZR0SR, FZR1SR, FZR2SR, FZR5SR, FZR10SR, SLT0SR, SLT10SR, SLT20SR, SLT40SR
except:
  pass
try:
  del SPSFile
except:
  pass